**STEP 1**: The installation, importation and upgrade of the necessary libraries

In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from ipywidgets import widgets, VBox, HBox, Output

**STEP 2**: Creating the interactive UI

In [34]:
output = Output()

ticker_label = widgets.Label(value="Ticker Symbol:")
ticker_input = widgets.Text(placeholder="Enter ticker symbol", style={'description_width': 'initial'})

start_date_label = widgets.Label(value="Start Date:")
start_date_input = widgets.Text(placeholder="Enter start date", style={'description_width': 'initial'})

end_date_label = widgets.Label(value="End Date:")
end_date_input = widgets.Text(placeholder="Enter end date", style={'description_width': 'initial'})

risk_free_rate_label = widgets.Label(value="Risk-Free Rate:")
risk_free_rate_input = widgets.FloatText(placeholder="Enter end date", style={'description_width': 'initial'})

calculate_button = widgets.Button(description="Calculate Sharpe Ratio", button_style='success')

def calculate_sharpe(button):
    with output:
        output.clear_output()

        try:
            ticker = ticker_input.value.strip()
            start_date = datetime.strptime(start_date_input.value.strip(), "%Y-%m-%d")
            end_date = datetime.strptime(end_date_input.value.strip(), "%Y-%m-%d")
            risk_free_rate = risk_free_rate_input.value

            if start_date >= end_date:
                print("Error: Start date must be before end date.")
                return

            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"Error: No data found for ticker {ticker.upper()} in the given date range.")
                return

            data = data.drop(columns=['Adj Close', 'High', 'Low', 'Open', 'Volume'])
            date_dif = end_date - start_date
            years = date_dif.days / 365

            start_price = data['Close'].iloc[0]
            end_price = data['Close'].iloc[-1]

            annual_return = float((end_price / start_price) ** (1 / years)) - 1

            data['Daily Return'] = data['Close'].pct_change().fillna(0)
            annual_std_dev = data['Daily Return'].std() * np.sqrt(252)

            sharpe_ratio = (annual_return - risk_free_rate) / annual_std_dev

            print(f"\nThe evaluation results for {ticker.upper()} from {start_date.date()} to {end_date.date()}:")
            print(f"Annual Return: {annual_return:.2%}")
            print(f"Annualized Standard Deviation: {annual_std_dev:.2%}")
            print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

            plt.figure(figsize=(10, 6))
            plt.plot(data['Close'], label=f"{ticker.upper()} Closing Prices")
            plt.title(f"{ticker.upper()} Closing Prices")
            plt.xlabel("Date")
            plt.ylabel("Price")
            plt.legend()
            plt.grid()
            plt.show()

        except ValueError as e:
            print(f"Input Error: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

calculate_button.on_click(calculate_sharpe)

ui = VBox([
    HBox([ticker_label, ticker_input], layout=widgets.Layout(margin="15px 0")),
    HBox([start_date_label, start_date_input], layout=widgets.Layout(margin="15px 0")),
    HBox([end_date_label, end_date_input], layout=widgets.Layout(margin="15px 0")),
    HBox([risk_free_rate_label, risk_free_rate_input], layout=widgets.Layout(margin="15px 0")),
    calculate_button,
    output
])

display(ui)